<a href="https://colab.research.google.com/github/Jack3690/INSIST/blob/main/notebooks/Galaxy_Simulation_PISTA_GalSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install insist-pista --quiet
!pip install galsim --quiet

In [ ]:
!pip install GalSim --quiet

In [12]:
import pista as pt
import galsim
from pista.simulation import Imager
from astropy.io import fits

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
data_path = pt.data_dir

In [13]:
class Galaxy(Imager):
  """Class for simulating Spectra"""
  def __init__(self, df, coords=None, tel_params={}, n_x=1000,
                n_y=1000, exp_time=100, plot=False, user_profiles={},
                **kwargs):
    self.gal_width = n_x if n_x>=n_y else n_y
    super().__init__(df, coords, tel_params, n_x, n_y, exp_time, plot,
                      user_profiles, **kwargs)

  def generate_sim_field(self, plot):
      """This function creates array with FoV a bit wider
      than user defined size for flux conservation"""
      if self.df is not None:
          self.calc_zp(plot=plot)
          self.init_psf_patch()

          # Cropping df to sim_field
          x_left = self.n_pix_psf//2 + self.gal_width//2
          x_right = self.n_x_sim - self.n_pix_psf//2 - self.gal_width//2
          y_left = self.n_pix_psf//2 + self.gal_width//2
          y_right = self.n_y_sim - self.n_pix_psf//2 - self.gal_width//2

          self.sim_df = self.init_df(df=self.df,
                                      n_x=self.n_x_sim, n_y=self.n_y_sim,
                                      x_left=x_left, x_right=x_right,
                                      y_left=y_left, y_right=y_right)
          if len(self.sim_df) < 1:
              print("Not Enough sources inside FoV. Increase n_x\
                              and n_y")
      else:
          print("df cannot be None")

  def check_df(self):
      # Input Dataframe
      if 'mag' not in self.df.keys():
          raise Exception("'mag' column not found input dataframe")

      if 'ra' not in self.df or 'dec' not in self.df.keys():
          if 'x' in self.df.keys() and 'y' in self.df.keys():
              print("Converting xy to ra-dec")
              self.df.x = self.df.x + 2
              self.df.y = self.df.y + 2
              self.df = self.xy_to_radec(self.df, self.n_x, self.n_y,
                                          self.pixel_scale)
          else:
              raise Exception("'ra','dec','x',or 'y', \
                columns not found in input dataframe ")

  def init_psf_patch(self, return_psf=False):
      """Creates PSF array from NPY or fits files"""
      ext = self.psf_file.split('.')[-1]

      if ext == 'npy':
          image = np.load(self.psf_file)
      elif ext == 'fits':
          image = fits.open(self.psf_file)[0].data

      image /= image.sum()  # Flux normalized to 1
      self.psf = image

      self.n_pix_psf = self.psf.shape[0]

      # Defining shape of simulation field
      self.n_x_sim = self.n_x + 2*self.gal_width
      self.n_y_sim = self.n_y + 2*self.gal_width

      if return_psf:
          return image*self.zero_flux
  def generate_photons(self, image, patch_width, df):

    psf_array = self.psf
    psf = galsim.Image(psf_array.shape[0], psf_array.shape[1])
    psf.array[:,:] = psf_array

    psf = galsim.InterpolatedImage(psf, scale = self.pixel_scale)

    for i, row in df.iterrows():
      # Bulge
      flux  = self.zero_flux*10**(-0.4*row['bulge_M'])  # total counts on the image
      rhalf = row['bulge_Re']/self.pixel_scale

      gal = galsim.Sersic(row['bulge_n'],flux=flux, half_light_radius=rhalf)

      ellipticity = galsim.Shear(  q=row['bulge_ba'],
                                beta=row['bulge_pa']*galsim.degrees)

      gal = gal.shear(ellipticity)

      bulge = galsim.Convolve([gal, psf])

      # Bar
      flux  = self.zero_flux*10**(-0.4*row['bar_M'])  # total counts on the image
      rhalf = row['bar_Re']/self.pixel_scale # arcsec

      gal  = galsim.Sersic(row['bar_n'],flux=flux, half_light_radius=rhalf)

      ellipticity = galsim.Shear(   q=row['bar_ba'],
                                 beta=row['bar_pa']*galsim.degrees)
      gal = gal.shear(ellipticity)

      bar = galsim.Convolve([gal, psf])


      # Disk
      flux  = self.zero_flux*10**(-0.4*row['disk_M'])  # total counts on the image
      rhalf = row['disk_Re']/self.pixel_scale # arcsec

      gal     = galsim.Sersic(row['disk_n'],flux=flux, half_light_radius=rhalf)

      ellipticity = galsim.Shear(   q=row['disk_ba'],
                                 beta=row['disk_pa']*galsim.degrees)
      gal = gal.shear(ellipticity)

      disk   = galsim.Convolve([gal, psf])

      final = galsim.Add(bulge, bar, disk)
      bounds = galsim.BoundsI(1, self.gal_width, 1, self.gal_width)
      final = final.drawImage(scale=self.pixel_scale, bounds=bounds)

      galaxy_img = final.array

      x0 = int(row['x'])
      y0 = int(row['y'])

      x1 = x0 - self.gal_width//2
      x2 = x1 + self.gal_width
      y1 = y0 - self.gal_width//2
      y2 = y1 + self.gal_width

      image[y1:y2, x1:x2] += galaxy_img

    image = image[self.gal_width:-self.gal_width,
                  self.gal_width:-self.gal_width]
    return image

    def __call__(self, det_params=None, n_stack=1, stack_type='median',
                 photometry=None, fwhm=None, detect_sources=False, **kwargs):
        self.ZP = 1
        super().__call__(det_params=det_params, n_stack=n_stack,
                         photometry=None,
                         stack_type=stack_type, ZP=1,**kwargs)

In [19]:
# Coma cluster properties
cluster_center_ra = 194.9558  # RA of the Coma cluster center
cluster_center_dec = 27.9807  # Dec of the Coma cluster center
cluster_size_arcsec = 20/60 # Size of the Coma cluster in arcseconds
num_galaxies = 100  # Number of galaxies to generate

# Generate galaxy positions using a Gaussian distribution
np.random.seed(42)

ra = np.random.normal(cluster_center_ra, cluster_size_arcsec / np.sqrt(8*np.log(2)), num_galaxies)
dec = np.random.normal(cluster_center_dec, cluster_size_arcsec / np.sqrt(8*np.log(2)), num_galaxies)
mag = np.random.uniform(16, 20, num_galaxies)

# Generate parameters for bulge, bar, and disk components
bulge_M = np.random.uniform(18, 19, num_galaxies)
bulge_Re = np.random.uniform(0.2, 1.0, num_galaxies)
bulge_n = np.random.uniform(1.0, 4.0, num_galaxies)
bulge_ba = np.random.uniform(0.5, 0.9, num_galaxies)
bulge_pa = np.random.uniform(0, 180, num_galaxies)

bar_M = np.random.uniform(17, 18, num_galaxies)
bar_Re = np.random.uniform(1.0, 3.0, num_galaxies)
bar_n = np.random.uniform(0.4, 1.0, num_galaxies)
bar_ba = np.random.uniform(0.3, 0.7, num_galaxies)
bar_pa = np.random.uniform(0, 180, num_galaxies)

disk_M = np.random.uniform(16, 17, num_galaxies)
disk_Re = np.random.uniform(2.0, 4.0, num_galaxies)
disk_n = np.random.uniform(0.8, 1.2, num_galaxies)
disk_ba = np.random.uniform(0.6, 0.95, num_galaxies)
disk_pa = np.random.uniform(0, 180, num_galaxies)

# Create the DataFrame for galaxy cluster
df = pd.DataFrame({'ra': ra, 'dec': dec, 'mag': mag,
                   'bulge_M': bulge_M, 'bulge_Re': bulge_Re, 'bulge_n': bulge_n,
                   'bulge_ba': bulge_ba, 'bulge_pa': bulge_pa,
                   'bar_M': bar_M, 'bar_Re': bar_Re, 'bar_n': bar_n,
                   'bar_ba': bar_ba, 'bar_pa': bar_pa,
                   'disk_M': disk_M, 'disk_Re': disk_Re, 'disk_n': disk_n,
                   'disk_ba': disk_ba, 'disk_pa': disk_pa})

In [20]:
tel_params ={
            'aperture'       : 100,
            'pixel_scale'    : 0.1,
            'psf_file'       : f'{data_path}/PSF/INSIST/off_axis_poppy.npy',
            'response_funcs' :  [ f'{data_path}/INSIST/UV/Coating.dat,5,100',   # 6 mirrors
                                  f'{data_path}/INSIST/UV/Filter.dat,1,100',
                                  f'{data_path}/INSIST/UV/Dichroic.dat,2,100',   # 2 dichroics
                                ],
             'coeffs'       : 1,
             'theta'        : 0
            }

In [21]:
sim = Galaxy(df = df, coords = (cluster_center_ra, cluster_center_dec),
             tel_params = tel_params, exp_time = 2400, plot = False,
             n_x = 6000, n_y = 6000)

In [ ]:
sim.show_field()

In [23]:
det_params = {'shot_noise' :  'Poisson',
              'qe_response': [],# [f'{data_path}/INSIST/UV/QE.dat,1,100'],
              'qe_mean'    : 0.95,
              'G1'         :  1,
              'bias'       : 10,
              'PRNU_frac'  :  0.25/100,
              'DCNU'       :  0.1/100,
              'RN'         :  3,
              'T'          :  218,
              'DN'         :  0.01/100
              }

In [ ]:
sim(det_params=det_params)

In [ ]:
sim.show_image()